In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

# Import from your codebase
from CLAPWrapper import CLAPWrapper
from datasets.esc50 import ESC50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [2]:
print("\n" + "="*80)
print("Loading ESC50 Dataset")
print("="*80)

root_path = "./data"
dataset = ESC50(root=root_path, download=True)
print(f"✅ Dataset loaded: {len(dataset)} samples")
print(f"   Classes: {len(dataset.classes)} categories")
print(f"   Sample classes: {dataset.classes[:5]}")

# Prepare text prompts
prompt = 'this is the sound of '
text_labels = [prompt + x for x in dataset.classes]
print(f"\n📝 Text prompts: {len(text_labels)} classes")


Loading ESC50 Dataset
Loading audio files


0it [00:00, ?it/s]

2000it [00:00, 16849.47it/s]

✅ Dataset loaded: 2000 samples
   Classes: 50 categories
   Sample classes: ['airplane', 'breathing', 'brushing teeth', 'can opening', 'car horn']

📝 Text prompts: 50 classes


In [3]:
# ============================================================================
# STEP 3: Initialize Models
# ============================================================================
print("\n" + "="*80)
print("Initializing Models")
print("="*80)

# Residual config con pc_weights = 1.0 (identità)
residual_config = {
    'n_components_ratio': .1,
    'reweight_factor': 2.0,
    'target_layers': [0, 1, 2, 3],  # Layers dove applicare reweighting
    'analysis_mode': True
}

print("\n🔧 Loading CLAP Standard...")
clap_standard = CLAPWrapper(
    version='2023',  # or '2022'
    use_cuda=torch.cuda.is_available(),
    type='classic'
)

print("\n🔧 Loading ResiDualCLAP...")
clap_residual = CLAPWrapper(
    version='2023',
    use_cuda=torch.cuda.is_available(),
    type='residual',
    residual_config=residual_config
)


Initializing Models

🔧 Loading CLAP Standard...

🔧 Loading ResiDualCLAP...
[2, 2, 6, 2]
🔍 Detecting layer dimensions...
  ✓ 0: torch.Size([1, 1024, 192])
  ✓ 1: torch.Size([1, 256, 384])
  ✓ 2: torch.Size([1, 64, 768])
  ✓ 3: torch.Size([1, 64, 768])
  ✓ layer_0: 192D → 19 PCs
  ✓ layer_1: 384D → 38 PCs
  ✓ layer_2: 768D → 76 PCs
  ✓ layer_3: 768D → 76 PCs


In [ ]:
print("\n" + "="*80)
print("Fitting PCA Components")
print("="*80)

# Prepare audio samples for PCA fitting
print("Collecting samples for PCA fitting (max 200 samples)...")

# Create a simple dataloader wrapper per PCA fitting
class SimpleAudioDataset:
    def __init__(self, wrapper, esc50_dataset, max_samples=1000):
        self.wrapper = wrapper
        self.audio_paths = []
        for i in range(min(max_samples, len(esc50_dataset))):
            audio_path, _, _ = esc50_dataset[i]
            self.audio_paths.append(audio_path)
    
    def __len__(self):
        return len(self.audio_paths)
    
    def __getitem__(self, idx):
        audio_tensor = self.wrapper.load_audio_into_tensor(
            self.audio_paths[idx],
            self.wrapper.args.duration,
            resample=True
        )
        # ✅ Assicurati sia 1D
        if audio_tensor.dim() > 1:
            audio_tensor = audio_tensor.squeeze()
        
        return audio_tensor

# Create dataset and loader
pca_dataset = SimpleAudioDataset(clap_residual, dataset, max_samples=50)
pca_loader = DataLoader(
    pca_dataset, 
    batch_size=16, 
    shuffle=False,
    num_workers=0,  # Start with 0 for debugging
    pin_memory=False
)

# Fit PCA
print(f"Fitting PCA on {len(pca_dataset)} samples...")
variance_ratios = clap_residual.clap.audio_encoder.base.htsat.fit_spectral_layers(
    pca_loader,
    max_samples=50
)

print("\n📊 PCA Variance Ratios:")
for layer_name, ratios in variance_ratios.items():
    print(f"   {layer_name}: Top 5 components = {ratios[:5]}")


Fitting PCA Components

🔬 PRE-FLIGHT CHECK:

🔍 Collecting data for rank check...


Collecting:   0%|          | 0/4 [00:00<?, ?it/s]


RANK ANALYSIS PER LAYER

LAYER_0: 51,200 samples × 192 features
  Rank numerico: 192
  Componenti richieste: 19
  ✓ OK: rank sufficiente
  Top 10 singular values:
     1: 3.07e+02 ██████████████████████████████
     2: 2.29e+02 ██████████████████████
     3: 1.82e+02 █████████████████
     4: 1.42e+02 █████████████
     5: 1.30e+02 ████████████
     6: 1.23e+02 ████████████
     7: 1.12e+02 ██████████
     8: 1.11e+02 ██████████
     9: 1.08e+02 ██████████
    10: 9.82e+01 █████████
  Varianza con 19 comp: 77.14%

LAYER_1: 12,800 samples × 384 features
  Rank numerico: 384
  Componenti richieste: 38
  ✓ OK: rank sufficiente
  Top 10 singular values:
     1: 3.56e+02 ██████████████████████████████
     2: 2.68e+02 ██████████████████████
     3: 2.44e+02 ████████████████████
     4: 1.95e+02 ████████████████
     5: 1.73e+02 ██████████████
     6: 1.54e+02 ████████████
     7: 1.36e+02 ███████████
     8: 1.28e+02 ██████████
     9: 1.23e+02 ██████████
    10: 1.15e+02 █████████
  Varia


✓ Collection completed in 11.03s
  • Total samples processed: 50
  • Successful batches: 4
  • Failed batches: 0
  • Samples per second: 4.5

📦 Collected data per layer:
  • layer_0: 4 batches, 51,200 tokens
  • layer_1: 4 batches, 12,800 tokens
  • layer_2: 4 batches, 3,200 tokens
  • layer_3: 4 batches, 3,200 tokens

📊 PHASE 2: Fitting PCA Components


Fitting PCA:  25%|██▌       | 1/4 [00:00<00:00,  8.53layer/s]

...

  • PCA completed in 0.10s

  📈 Variance Analysis:
     • Total components: 38
     • Top 5 variances: ['0.1763', '0.1089', '0.0752', '0.0540', '0.0490']
     • Cumulative variance:
        - 50% variance: 6/38 components (15.8%)
        - 70% variance: 16/38 components (42.1%)
        - 80% variance: 28/38 components (73.7%)
        - 90% variance: 39/38 components (102.6%)
        - 95% variance: 39/38 components (102.6%)

     Top 10 components bar:
        PC 1: ████████████████████████████████████████ 0.1763
        PC 2: ████████████████████████ 0.1089
        PC 3: █████████████████ 0.0752
        PC 4: ████████████ 0.0540
        PC 5: ███████████ 0.0490
        PC 6: █████████ 0.0406
        PC 7: ██████ 0.0308
        PC 8: ██████ 0.0276
        PC 9: █████ 0.0241
        PC10: █████ 0.0231

🔧 Processing layer_2:
  • Concatenating 4 batches...
  • Combined shape: torch.Size([3200, 768]) (3,200 samples)
  • Memory usage: 9.4 MB
  • Running PCA decomposition...


Fitting PCA:  75%|███████▌  | 3/4 [00:01<00:00,  1.16layer/s]

  • PCA completed in 1.70s

  📈 Variance Analysis:
     • Total components: 76
     • Top 5 variances: ['0.2302', '0.0595', '0.0550', '0.0432', '0.0343']
     • Cumulative variance:
        - 50% variance: 8/76 components (10.5%)
        - 70% variance: 26/76 components (34.2%)
        - 80% variance: 46/76 components (60.5%)
        - 90% variance: 77/76 components (101.3%)
        - 95% variance: 77/76 components (101.3%)

     Top 10 components bar:
        PC 1: ████████████████████████████████████████ 0.2302
        PC 2: ██████████ 0.0595
        PC 3: █████████ 0.0550
        PC 4: ███████ 0.0432
        PC 5: █████ 0.0343
        PC 6: █████ 0.0305
        PC 7: ████ 0.0240
        PC 8: ████ 0.0234
        PC 9: ███ 0.0200
        PC10: ██ 0.0168

🔧 Processing layer_3:
  • Concatenating 4 batches...
  • Combined shape: torch.Size([3200, 768]) (3,200 samples)
  • Memory usage: 9.4 MB
  • Running PCA decomposition...


Fitting PCA: 100%|██████████| 4/4 [00:03<00:00,  1.23layer/s]

  • PCA completed in 1.16s

  📈 Variance Analysis:
     • Total components: 76
     • Top 5 variances: ['0.1211', '0.0795', '0.0751', '0.0657', '0.0530']
     • Cumulative variance:
        - 50% variance: 8/76 components (10.5%)
        - 70% variance: 16/76 components (21.1%)
        - 80% variance: 24/76 components (31.6%)
        - 90% variance: 44/76 components (57.9%)
        - 95% variance: 77/76 components (101.3%)

     Top 10 components bar:
        PC 1: ████████████████████████████████████████ 0.1211
        PC 2: ██████████████████████████ 0.0795
        PC 3: ████████████████████████ 0.0751
        PC 4: █████████████████████ 0.0657
        PC 5: █████████████████ 0.0530
        PC 6: ███████████████ 0.0476
        PC 7: █████████████ 0.0397
        PC 8: ████████████ 0.0366
        PC 9: ███████████ 0.0344
        PC10: ██████████ 0.0331

✓ PCA Fitting Completed Successfully
Total time: 14.29s

Spectral layers ready for reweighting:
  • layer_0: ✓ FITTED
  • layer_1: ✓ F

In [14]:
# Dopo aver fatto la collection
for layer_name, outputs in collected_outputs.items():
    if outputs:
        X = torch.cat(outputs, dim=0)
        print(f"\n{'='*60}")
        print(f"Checking {layer_name}")
        quick_rank_check(X)

NameError: name 'collected_outputs' is not defined

In [5]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_standard.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 100  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_baseline, y_labels = [], []

for i in tqdm(range(test_size), desc="Baseline"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_standard.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_standard.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_baseline.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_baseline_array = np.concatenate(y_preds_baseline, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_baseline_array, axis=1)
)

print(f"\n✅ Baseline Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 100 samples...


Baseline:   0%|          | 0/100 [00:00<?, ?it/s]


✅ Baseline Accuracy: 0.920 (92.0%)


In [6]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_residual.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 100  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_residual, y_labels = [], []

for i in tqdm(range(test_size), desc="residual"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_residual.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_residual.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_residual.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_residual_array = np.concatenate(y_preds_residual, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_baseline_array, axis=1)
)

print(f"\n✅ Residual Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 100 samples...


residual:   0%|          | 0/100 [00:00<?, ?it/s]


✅ Residual Accuracy: 0.920 (92.0%)


In [7]:
audio_path, target, one_hot_target = dataset[0]

In [8]:
clap_residual.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.8840,  0.3185, -0.6708,  ...,  2.1432,  1.0298, -0.1217]])

In [9]:
clap_standard.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.7623,  0.2343, -0.5101,  ...,  1.8940,  0.9414, -0.0119]])